In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn import svm
from sklearn import metrics
import scipy.io
from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats, linalg
import warnings
import math
import random
from sklearn.exceptions import ConvergenceWarning
from statsmodels.stats.multitest import multipletests
def conv_r2z(r):
    with np.errstate(invalid='ignore', divide='ignore'):
        return 0.5 * (np.log(1 + r) - np.log(1 - r))
def conv_z2r(z):
    with np.errstate(invalid='ignore', divide='ignore'):
        return (np.exp(2 * z) - 1) / (np.exp(2 * z) + 1)
os.chdir('/gpfs/milgram/project/chun/jk2992/rest_thoughts/') # change to your folder path

# Read in brain data

In [2]:
def reshape_FC(fc):
    fc = np.transpose(fc,(2,0,1))
    fc = np.reshape(fc,(fc.shape[0],fc.shape[1]*fc.shape[2]))
    return fc

In [3]:
rest_FC = scipy.io.loadmat('./data/brain/rest_fc.mat')['rest'][0]
# delete 1032 s1r1 (response box error), which is index 29 because there is no 1004 and 1019
rest_FC[29] = np.delete(rest_FC[29],[0],axis = 0)
print('FC profiles shape: '+str(len(rest_FC))+'*'+str(rest_FC[2].shape))

# calculate FC by subject
nsubj = len(rest_FC)
FC_bysub = []
for i in range(nsubj):
    FC_bysub.append(reshape_FC(rest_FC[i]))

FC profiles shape: 60*(4, 8, 35778)


# Read in behavioral data (e.g., awake)

In [4]:
def get_num(string):
    num = []
    for i in range(len(string)):
        if i % 5 == 2:
            num.append(int(string[i]))
    return num

In [5]:
path = './data/beh/'
df = pd.read_csv(path + 'all_ratings.csv')
# create a behavioral participant list
beh_list = np.unique(df['Sub'])
print('We have '+str(len(beh_list)) + ' participants')

# create a behavioral dataset by subject
idx = -1
beh_bysub = []
for sub in range(len(beh_list)):
    sub_data = df[df['Sub']==beh_list[sub]]['Awake']
    sub_vec = []
    # print(str(beh_list[sub]) + ': ' + str(len(sub_data)*8))
    for run in range(len(sub_data)):
        idx = idx + 1
        run_data = get_num(sub_data[idx])
        sub_vec.append(run_data)
    sub_vec = np.asarray(sub_vec)
    sub_vec = np.reshape(sub_vec,(sub_vec.shape[0]*sub_vec.shape[1]))
    # zscore the ratings
    sub_vec = stats.zscore(sub_vec)
    beh_bysub.append(sub_vec)

# remove 1032 s1r1 (response box error), 1044 s1r2 (brain data converting error)
beh_bysub[29] = np.delete(beh_bysub[29],range(0,8))
beh_bysub[41] = np.delete(beh_bysub[41],range(8,16))

# let's double check the brain match behavior
count = 0
for i in range(nsubj):
    if len(beh_bysub[i]) == FC_bysub[i].shape[1]:
        count = count + 1
if count == nsubj:
    print('All behavioral and brain data match')

We have 60 participants
All behavioral and brain data match


# select clean data for model building

In [6]:
good_trial_bysub = []
good_trial_id_bysub = []
good_sub = []
for sub in range(nsubj):
    sub_data = FC_bysub[sub]
    count = 0 
    good_trial_id = []
    for trial in range(sub_data.shape[1]):
        # a good trial has < 1000 missing FC (3 missing nodes)
        if np.sum(np.isnan(sub_data[:,trial])) < 1000:
            count = count + 1
            good_trial_id.append(trial)
    
    good_trial_bysub.append(count)
    # a good participant has > 20 good trials
    if count > 20:
        good_sub.append(sub)
        good_trial_id = np.transpose(good_trial_id)
        good_trial_id_bysub.append(good_trial_id)

good_sub = np.transpose(good_sub)        
## delete sub-1040 because the rating for future and past are constant
# good_sub = np.delete(good_sub,[30])
# good_trial_id_bysub.pop(30)

# print('we have ' + str(np.sum(good_trial_bysub)) + ' good trials in total')
print('We have ' + str(len(good_sub)) + ' good participants (>20 good quality trials in brain data). They are:')
print(' ')
print(beh_list[good_sub])

We have 50 good participants (>20 good quality trials in brain data). They are:
 
[1003 1005 1006 1007 1009 1011 1012 1013 1014 1016 1017 1018 1021 1022
 1023 1024 1025 1026 1027 1028 1029 1030 1031 1032 1033 1034 1036 1037
 1038 1039 1040 1041 1042 1043 1044 1045 1046 1047 1048 1049 1050 1051
 1052 1055 1056 1057 1059 1060 1061 1062]


In [7]:
# select good participants
nsubj_good = len(good_sub)
FC_selected, beh_selected = [], []
for i in range(nsubj_good):
    tmp_FC = FC_bysub[good_sub[i]]
    tmp_FC = tmp_FC[:,good_trial_id_bysub[i]]
    FC_selected.append(tmp_FC)
    
    tmp_beh = beh_bysub[good_sub[i]]
    tmp_beh = tmp_beh[good_trial_id_bysub[i]]    
    beh_selected.append(tmp_beh)

# let's double check the brain match behavior
count = 0
count_trial = []
for i in range(nsubj_good):
    if len(beh_selected[i]) == FC_selected[i].shape[1]:
        count = count + 1
        count_trial.append(len(beh_selected[i]))
if count == nsubj_good:
    print('We have ' + str(np.sum(count_trial)) + ' good trials in these good participants')
    print('All behavioral and brain data match after selecting good participants')

We have 1531 good trials in these good participants
All behavioral and brain data match after selecting good participants


# Compute correlation matrix (part of feature selection)

In [8]:
def get_r_omit_nan(tc1,tc2):
    # get nan position
    nanidx = np.where(np.isnan(tc1))

    if len(tc1) - len(nanidx[0]) > 12:
        # remove nan from both tc
        tc1 = np.delete(tc1,nanidx)
        tc2 = np.delete(tc2,nanidx)
        # run correlation
        rval = stats.pearsonr(tc1,tc2)[0]
    else:
        rval = np.nan
    return rval

In [13]:
nR = 268
nFC = int(nR*(nR-1)/2)
corrmat = np.zeros((nsubj_good,nFC))
for sub in range(nsubj_good):
    print('Running sub ' + str(sub+1) + '/' + str(nsubj_good))
    for feat in range(nFC):
        corrmat[sub,feat] = get_r_omit_nan(FC_selected[sub][feat,:],beh_selected[sub])

Running sub 1/50
Running sub 2/50
Running sub 3/50
Running sub 4/50
Running sub 5/50
Running sub 6/50
Running sub 7/50
Running sub 8/50
Running sub 9/50
Running sub 10/50
Running sub 11/50
Running sub 12/50
Running sub 13/50
Running sub 14/50
Running sub 15/50
Running sub 16/50
Running sub 17/50
Running sub 18/50
Running sub 19/50
Running sub 20/50
Running sub 21/50
Running sub 22/50
Running sub 23/50
Running sub 24/50
Running sub 25/50
Running sub 26/50
Running sub 27/50
Running sub 28/50
Running sub 29/50
Running sub 30/50
Running sub 31/50
Running sub 32/50
Running sub 33/50
Running sub 34/50
Running sub 35/50
Running sub 36/50
Running sub 37/50
Running sub 38/50
Running sub 39/50
Running sub 40/50
Running sub 41/50
Running sub 42/50
Running sub 43/50
Running sub 44/50
Running sub 45/50
Running sub 46/50
Running sub 47/50
Running sub 48/50
Running sub 49/50
Running sub 50/50


# Computational modeling

In [14]:
def concate(mat,ax,concate_range):
    concate_mat = np.concatenate((mat[concate_range[0]],mat[concate_range[1]]), axis = ax)
    for i in concate_range[2:]:
        concate_mat = np.concatenate((concate_mat,mat[i]),axis = ax)
    return concate_mat

In [15]:
def losocv(fmri,beh,nsubj,corr_mat):
    output_acc, output_eval, output_pos_feat, output_neg_feat = [], [], np.zeros((nsubj, nR, nR)), np.zeros((nsubj, nR, nR))
    for test_sub in range(nsubj):
        # separate train-test subject
        range_train = np.delete(range(nsubj),test_sub)
        test_fmri = fmri[test_sub]
        train_fmri = concate(fmri,1,range_train)
        
        test_beh = beh[test_sub]
        train_beh = concate(beh,0,range_train)
        
        # features: correlation with behavior
        train_subj_corrmat = np.delete(corr_mat,test_sub,0)
        
        # feature selection
        pos_feat, neg_feat, all_feat, nanval = [], [], [], 0
        idx = -1
        for feat in range(int(nR*(nR-1)/2)):
            idx=idx+1
                
            if np.any(np.isnan(train_subj_corrmat[:,feat])) or np.any(np.isnan(test_fmri[feat,:])) or np.any(np.isnan(train_fmri[feat,:])):
                nanval = nanval+1
                pass
            else:
                [tval,pval] = scipy.stats.ttest_1samp(train_subj_corrmat[:,feat],0)
                if pval < thres:
                    if np.average(train_subj_corrmat[:,feat])>0:
                        all_feat.append(idx)
                        pos_feat.append(idx)
                    elif np.average(train_subj_corrmat[:,feat])<0:
                        all_feat.append(idx)
                        neg_feat.append(idx)
        pos_feat, neg_feat, all_feat = np.asarray(pos_feat), np.asarray(neg_feat), np.asarray(all_feat)
        
        # summarize feature matrix
        idx = -1
        for i1 in range(nR-1):
            for i2 in range(i1+1,nR):
                idx = idx+1
                if idx in pos_feat:
                    output_pos_feat[test_sub,i1,i2]=1
                    output_pos_feat[test_sub,i2,i1]=1
                if idx in neg_feat:
                    output_neg_feat[test_sub,i1,i2]=1
                    output_neg_feat[test_sub,i2,i1]=1

        train_fmri_selected = train_fmri[all_feat,:]
        test_fmri_selected = test_fmri[all_feat,:]
        
        # Support vector regression with non-linear kernel
        clf = []
        clf = svm.SVR(kernel='rbf',max_iter=1000, gamma='auto')
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning)
            clf.fit(train_fmri_selected.T, train_beh)
        predicted = clf.predict(test_fmri_selected.T)
        output_acc.append(predicted)
        
        # evaluate
        pearsonr = scipy.stats.pearsonr(test_beh, predicted)[0]
        mse = metrics.mean_squared_error(test_beh, predicted)
        rsq = metrics.r2_score(test_beh, predicted)
        output_eval.append([pearsonr, mse, rsq])
        
        print('  subj '+str(test_sub+1)+' / '+str(nsubj)+': #feat '+str(len(all_feat))+', pearson r='+str(np.round(pearsonr,3)),', mse='+str(np.round(mse,3))+', rsq='+str(np.round(rsq,3)))
        print('               (train) ft '+str(train_fmri_selected.shape[0])+', beh '+str(train_beh.shape[0])+
              ', (test) ft '+str(test_fmri_selected.shape[0])+', beh '+str(test_beh.shape[0]))

    return output_acc, output_eval, output_pos_feat, output_neg_feat

In [16]:
def onetail_p(real, null):
    p = (1+np.sum(null>=real))/(1+len(null))
    print(str(np.sum(null>=real))+' among '+str(len(null))+' null has higher value than actual prediction')
    return p
def onetail_p_lower(real, null):
    p = (1+np.sum(null<=real))/(1+len(null))
    print(str(np.sum(null<=real))+' among '+str(len(null))+' null has lower value than actual prediction')
    return p
niter = 10000
def sig_test(predictions,actual,actual_r,actual_mse,actual_rsq):
    null_dist = np.zeros((nsubj_good,niter,3)) # sig test on 3 evaluation matrices: r, mse, rsq
    for j in range(nsubj_good):
        for i in range(niter):
            test_beh = actual[j]
            random.shuffle(test_beh)
            tmp_r = stats.pearsonr(test_beh,predictions[j])[0]
            tmp_mse = metrics.mean_squared_error(test_beh,predictions[j])
            tmp_rsq = metrics.r2_score(test_beh,predictions[j])
            null_dist[j,i,0] = tmp_r
            null_dist[j,i,1] = tmp_mse
            null_dist[j,i,2] = tmp_rsq
    final_output_r, final_output_mse, final_output_rsq = [], [], []
    for k in range(niter):
        tmp_rval = null_dist[:,k,0] # null rvals
        tmp_rval = conv_z2r(np.mean(conv_r2z(tmp_rval)))
        final_output_r.append(tmp_rval)
        tmp_mse = null_dist[:,k,1] # null mse
        tmp_mse = np.mean(tmp_mse)
        final_output_mse.append(tmp_mse)
        tmp_rsq = null_dist[:,k,2] # null rsq
        tmp_rsq = np.mean(tmp_rsq)
        final_output_rsq.append(tmp_rsq)
    final_output_r = np.asarray(final_output_r)
    final_output_mse = np.asarray(final_output_mse)
    final_output_rsq = np.asarray(final_output_rsq)
    pval_r = onetail_p(actual_r,final_output_r)
    print('rvals: p = '+str(pval_r))
    pval_mse = onetail_p_lower(actual_mse,final_output_mse)# because mse is the smaller the better
    print('mse: p = '+str(pval_mse))
    pval_rsq = onetail_p(actual_rsq,final_output_rsq)
    print('rsq: p = '+str(pval_rsq))
    return null_dist, pval_r, pval_mse, pval_rsq

In [17]:
thres = 0.05
output_acc, output_eval, output_pos_feat, output_neg_feat = losocv(FC_selected,beh_selected,nsubj_good,corrmat)

  subj 1 / 50: #feat 1708, pearson r=0.485 , mse=0.776, rsq=0.224
               (train) ft 1708, beh 1499, (test) ft 1708, beh 32
  subj 2 / 50: #feat 1782, pearson r=0.139 , mse=1.058, rsq=-0.058
               (train) ft 1782, beh 1499, (test) ft 1782, beh 32
  subj 3 / 50: #feat 1725, pearson r=0.162 , mse=1.079, rsq=-0.023
               (train) ft 1725, beh 1502, (test) ft 1725, beh 29
  subj 4 / 50: #feat 1824, pearson r=-0.237 , mse=1.21, rsq=-0.21
               (train) ft 1824, beh 1499, (test) ft 1824, beh 32
  subj 5 / 50: #feat 1771, pearson r=0.24 , mse=1.067, rsq=-0.067
               (train) ft 1771, beh 1499, (test) ft 1771, beh 32
  subj 6 / 50: #feat 1746, pearson r=0.122 , mse=1.092, rsq=-0.064
               (train) ft 1746, beh 1500, (test) ft 1746, beh 31
  subj 7 / 50: #feat 1751, pearson r=0.185 , mse=0.996, rsq=0.004
               (train) ft 1751, beh 1499, (test) ft 1751, beh 32
  subj 8 / 50: #feat 1749, pearson r=0.138 , mse=1.061, rsq=-0.061
             

In [18]:
# significance testing with permutation (shuffling behavior for 10000 times)
output_eval = np.asarray(output_eval)
# fig, ax = plt.subplots(dpi = 800)
# ax.hist(output_eval[:,0], color='gray')
# ax.spines[['right', 'top']].set_visible(False)
# plt.axvline(conv_z2r(np.mean(conv_r2z(output_eval[:,0]))),color = 'black')
print('Mean = ' + str(conv_z2r(np.mean(conv_r2z(output_eval[:,0])))) + ', STD = '+ str(np.std(output_eval[:,0])))
null_dist, p1, p2, p3 = sig_test(output_acc,beh_selected,conv_z2r(np.mean(conv_r2z(output_eval[:,0]))),np.mean(output_eval[:,1]),np.mean(output_eval[:,2]))

Mean = 0.07738535921359382, STD = 0.2329840465404623
22 among 10000 null has higher value than actual prediction
rvals: p = 0.0022997700229977
15 among 10000 null has lower value than actual prediction
mse: p = 0.0015998400159984002
15 among 10000 null has higher value than actual prediction
rsq: p = 0.0015998400159984002


In [ ]:
savepath = './results/CPMs/'
if not os.path.exists(savepath):
    os.makedirs(savepath)
# results = {'acc':output_acc, 'eval':output_eval,'p_r':p1, 'p_mse':p2, 'p_rsq': p3}
# scipy.io.savemat(savepath+'a_awake.mat',results)

# # save the null to plot the fancy figure using step05_plot_SVR.R
# results = {'acc':output_acc, 'eval':output_eval,'p_r':p1, 'p_mse':p2, 'p_rsq': p3, 'null':null_dist}
# scipy.io.savemat(savepath+'a_awake_null.mat',results)

# # save the selected features in each round of cross-validation to replicate the FC networks or for out-of-sample prediction
# not saving these results for now due to file size limit on github (for sharing). Please run this for your own analysis
results = {'acc':output_acc, 'eval':output_eval,'p_r':p1, 'p_mse':p2, 'pos_feat':output_pos_feat, 'neg_feat':output_neg_feat}
scipy.io.savemat(savepath+'a_awake_features.mat',results)

## repeat the same procedure for every thought dimension.

# Correct for multiple comparison

In [20]:
## run this after you finished running all 9 dimensions
savepath = './results/CPMs/'
vars = ['a_awake','b_external','c_future','d_past','e_other','f_valence','g_image','h_word','i_detail']
p_values = []
for var in vars:
    results = scipy.io.loadmat(savepath + var + '.mat')
    p_r, p_mse, p_rsq = results['p_r'][0][0], results['p_mse'][0][0], results['p_rsq'][0][0]
    p_values.append(np.array([p_r, p_mse, p_rsq]))
p_values = np.array(p_values)

# let's use a strict method for correcting multiple comparison
decision, adj_pvals, sidak_aplha, bonf_alpha  = multipletests(p_values[:,0],alpha = .01, method = 'bonferroni')
corrected_pvals = [round(i,4) for i in adj_pvals]
print('pvals for r:',corrected_pvals)
print(' ')

decision, adj_pvals, sidak_aplha, bonf_alpha  = multipletests(p_values[:,1],alpha = .01, method = 'bonferroni')
corrected_pvals = [round(i,4) for i in adj_pvals]
print('pvals for mse:',corrected_pvals)
print(' ')

decision, adj_pvals, sidak_aplha, bonf_alpha  = multipletests(p_values[:,2],alpha = .01, method = 'bonferroni')
corrected_pvals = [round(i,4) for i in adj_pvals]
print('pvals for rsq:',corrected_pvals)
print(' ')

pvals for r: [0.0207, 0.0027, 0.0009, 1.0, 0.441, 0.0018, 0.0027, 1.0, 1.0]
 
pvals for mse: [0.0144, 0.0009, 0.0009, 1.0, 0.3276, 0.0018, 0.0054, 1.0, 1.0]
 
pvals for rsq: [0.0144, 0.0009, 0.0009, 1.0, 0.3699, 0.0009, 0.0045, 1.0, 1.0]
 


# Create a clean result file to plot a beautiful figure in R (step04_plot_SVR.R)

In [21]:
def average_null(null,measure): # measure: 0=rvals, 1=mse, 2=rsq
    new_null = []
    for iter in range(niter):
        this_iter = null[:,iter,measure]
        if measure == 0:
            avg = conv_z2r(np.mean(conv_r2z(this_iter)))
        else:
            avg = np.mean(this_iter)
        new_null.append(avg)
    return np.array(new_null)

In [22]:
measures = ['r','mse','rsq']
for mi, measure in enumerate(measures):
    actual_vals, actual_labels = np.array([]), np.array([])
    null_vals, null_labels = np.array([]), np.array([])
    for var in vars:
        results = scipy.io.loadmat(savepath + var + '_null.mat')
        ## load the actual values
        vals = results['eval'][:,0]
        label_vec = [var]*len(vals) # create the labels
        actual_vals = np.concatenate((actual_vals,vals))
        actual_labels = np.concatenate((actual_labels,label_vec))
        ## load the null
        null = average_null(results['null'],0)
        null_vals = np.concatenate((null_vals,null))
        label_vec = [var]*niter # create the labels
        null_labels = np.concatenate((null_labels,label_vec))

    # saving the actual results
    df_actual = pd.DataFrame({
        'Group':np.array(actual_labels).flatten(),
        'vals':np.array(actual_vals).flatten()})
    # saving the null
    df_null = pd.DataFrame({
        'Group':np.array(null_labels).flatten(),
        'vals':np.array(null_vals).flatten()})

    df_actual.to_csv(savepath+measure+'_actual.csv',index=False)
    df_null.to_csv(savepath+measure+'_null.csv',index=False)